# Turn aggregated data into weekly-behavior data

In [129]:
import pandas as pd
import numpy as np
import re

## Transform Zoom dataset

In [2]:
zoom = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\student-dropout-analysis\data_processed\final_zoom.xlsx", dtype={'SID':str})
zoom

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,Grade,SID_full,Name,EFL,Note,MVA/MVSM,concat,4 last digits,Học sinh cùng lớp có chung 4 số cuối ID,_merge
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,Grade 1,2300-1816,Mai Nguyễn Trúc Linh,EFL K5,Chính thức,MVA,MVA2300-1816,1816.0,NaN,both
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,Grade 1,2200-2469,Nguyễn Phạm Nhật Minh,EFL K5,Chính thức,MVA,MVA2200-2469,2469.0,NaN,both
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,Grade 1,2400-2190,VY HÀ TÚ YÊN,EFL K5,Chính thức,MVA,MVA2400-2190,2190.0,NaN,both
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Lai Ngan Khanh 01771,23001771@minhvietacademy.org,...,Grade 1,2300-1771,Lại Ngân Khánh,EFL K5,Chính thức,MVA,MVA2300-1771,1771.0,NaN,both
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Tran Minh Khang 03927,23003927@minhvietacademy.org,...,Grade 1,2300-3927,Trần Minh Khang,EFL K5,Chính thức,MVA,MVA2300-3927,3927.0,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271177,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Hung, Viet",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271178,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Than Thu, Nguyen Minh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271179,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Tien, Anh",NaN,...,Grade 9-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
271180,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,phạm nguyên phúc hưng 02068,NaN,...,Grade 9-12,2400-2068,Phạm Nguyên Phúc Hưng,EFL 69,Chính thức,MVA,MVA2400-2068,2068.0,NaN,both


#### Aggregrate disconnected sessions
As students sometimes have internet issues and got disconnected from zoom, their participation is splitted into smaller sessions/records that seem like disengagement. So here we group them back together

In [4]:
#zoom events (ze)
ze = zoom.copy()

ze['Start time'] = pd.to_datetime(ze['Start time'], errors='coerce')
ze['End time'] = pd.to_datetime(ze['End time'], errors='coerce')
ze['Join time'] = pd.to_datetime(ze['Join time'], errors='coerce')
ze['Leave time'] = pd.to_datetime(ze['Leave time'], errors='coerce')


ze = (ze.groupby(['SID', 'Topic', 'Start time'])
              .agg(
                  end_time=('End time', 'max'),
                  student_duration=('student_stay_duration', 'sum'),
                  join_time=('Join time', 'min'),
                  leave_time=('Leave time', 'max'),
                  grade=('Grade', 'last'),
                  duration=('Duration (minutes)', 'first')
              )).reset_index()

ze['date'] = ze['Start time'].dt.floor("D")

ze

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17
...,...,...,...,...,...,...,...,...,...,...
207563,25000506,2501HUM,2025-05-27 19:25:00,2025-05-27 20:25:00,30,2025-05-27 19:55:00,2025-05-27 20:25:00,Grade 1,60,2025-05-27
207564,25000506,2501HUM,2025-05-29 19:27:00,2025-05-29 20:25:00,55,2025-05-29 19:30:00,2025-05-29 20:25:00,Grade 1,59,2025-05-29
207565,25000506,2501MAT,2025-05-14 19:26:00,2025-05-14 20:24:00,58,2025-05-14 19:26:00,2025-05-14 20:24:00,Grade 1,58,2025-05-14
207566,25000506,2501MAT,2025-05-21 19:26:00,2025-05-21 20:28:00,60,2025-05-21 19:28:00,2025-05-21 20:28:00,Grade 1,62,2025-05-21


#### Compute Minutes late / Minutes leaving early / Absent flag

In [38]:
grace_min = 5 #the school allows for 5 minutes grace

ze['minutes_late'] = ((ze['join_time'] - ze['Start time']).dt.total_seconds()/60) - grace_min
ze['minutes_late'] = ze['minutes_late'].clip(lower=0)

ze['minutes_left_early'] = ((ze['end_time'] - ze['leave_time']).dt.total_seconds()/60)
ze['minutes_left_early'] = ze['minutes_left_early'].clip(lower=0)

ze['attended_ratio'] = (ze['student_duration']/ze['duration'])
ze['very_short_flag'] = (ze['student_duration']<15).astype(int)
ze['partial_present'] = (ze['attended_ratio'] < 0.75).astype(int)
ze['is_present'] = (ze['attended_ratio']>0.25).astype(int)

ze.head()

,SID,Topic,Start time,end_time,student_duration,join_time,leave_time,grade,duration,date,minutes_late,minutes_left_early,attended_ratio,is_present,very_short_flag,partial_present
0,00000006,2506ELA,2025-02-24 18:24:00,2025-02-24 19:26:00,61,2025-02-24 18:24:00,2025-02-24 19:24:00,Grade 6,63,2025-02-24,0.0,2.0,0.968254,1,0,0
1,00000006,2506ELA,2025-02-26 18:25:00,2025-02-26 19:26:00,45,2025-02-26 18:42:00,2025-02-26 19:26:00,Grade 6,62,2025-02-26,12.0,0.0,0.725806,1,0,1
2,00000006,2506ELA,2025-03-03 18:26:00,2025-03-03 19:26:00,60,2025-03-03 18:27:00,2025-03-03 19:26:00,Grade 6,61,2025-03-03,0.0,0.0,0.983607,1,0,0
3,00000006,2506ELA,2025-03-05 18:25:00,2025-03-05 19:27:00,9,2025-03-05 19:16:00,2025-03-05 19:25:00,Grade 6,63,2025-03-05,46.0,2.0,0.142857,0,1,1
4,00000006,2506ELA,2025-03-17 18:27:00,2025-03-17 19:26:00,59,2025-03-17 18:27:00,2025-03-17 19:26:00,Grade 6,60,2025-03-17,0.0,0.0,0.983333,1,0,0


#### Aggregrate to day-level
As students can have multiple sessions per day, here we aggregrate to form the basis for week-level

In [52]:
day = (ze.groupby(['SID', 'date'])
       .agg(
           late_max=('minutes_late', 'max'), #max minutes late in that day
           late_mean=('minutes_late', 'mean'), #mean minutes late (over all the sessions in that day)
           late_sum=('minutes_late', 'sum'), #total minutes late in that day
           early_max=('minutes_left_early', 'max'), #max minutes left_early in that day
           early_mean=('minutes_left_early', 'mean'),#mean minutes left early (over all the sessions in that day)
           early_sum=('minutes_left_early', 'sum'), #total minutes left early in that day
           very_short_any=('very_short_flag', 'max'), #if there is 1 session attended under 15 minutes in that day or not 
           very_short_mean=('very_short_flag', 'mean'), #mean sessions atteneded under 15 minutes in that day
           very_short_sum=('very_short_flag', 'sum'), #total sessions atteneded under 15 minutes in that day
           partial_any=('partial_present', 'max'), #if there is 1 partially attended in that day or not 
           partial_mean=('partial_present', 'mean'), #mean sessions partially atteneded in that day
           partial_sum=('partial_present', 'sum'), #total sessions partially atteneded in that day
           attended_ratio_mean=('attended_ratio', 'mean'), #mean attended rate in that day
           attended_ratio_min=('attended_ratio', 'min'), #lowest attended rate in that day
           minutes_attended_day=('student_duration', 'sum'), #total minutes joined in that day
           total_duration_day=('duration', 'sum'), #total scheduled minutes of classes in that day
           is_present_day=('is_present', 'max')
       )).reset_index()

day['attended_ratio'] = day['minutes_attended_day']/day['total_duration_day'].replace(0,np.nan)
day.head()

,SID,date,late_max,late_mean,late_sum,early_max,early_mean,early_sum,very_short_any,very_short_mean,very_short_sum,partial_any,partial_mean,partial_sum,attended_ratio_mean,attended_ratio_min,minutes_attended_day,total_duration_day,is_present_day,attended_ratio
0,00000006,2025-02-20,0.0,0.0,0.0,14.0,7.0,14.0,0,0.0,0,1,0.5,1,0.872881,0.745763,106,121,1,0.876033
1,00000006,2025-02-24,0.0,0.0,0.0,11.0,6.5,13.0,0,0.0,0,0,0.0,0,0.885766,0.803279,110,124,1,0.887097
2,00000006,2025-02-25,3.0,1.5,3.0,7.0,4.5,9.0,0,0.0,0,0,0.0,0,0.860656,0.836066,105,122,1,0.860656
3,00000006,2025-02-26,12.0,6.0,12.0,17.0,8.5,17.0,0,0.0,0,1,1.0,2,0.707166,0.688525,87,123,1,0.707317
4,00000006,2025-02-27,0.0,0.0,0.0,20.0,20.0,20.0,0,0.0,0,1,1.0,1,0.590164,0.590164,36,61,1,0.590164


#### Convert day -> week statistics

In [80]:
to_week = lambda d:d.to_period("W-SUN").start_time
day['week_start'] = day['date'].map(to_week)

weekly_stats = (day.groupby(['SID', 'week_start'])
                .agg(
                    #Alert signs
                    late_max=('late_max', 'max'),
                    early_max=('late_max', 'max'),
                    total_late_minutes=('late_sum', 'sum'),
                    total_early_minutes=('early_sum', 'sum'),

                    #Trend
                    late_mean=('late_mean', 'mean'),
                    early_mean=('early_mean', 'mean'),
                    attended_ratio_mean=('attended_ratio_mean', 'mean'),
                    attended_ratio_week=('attended_ratio', 'mean'),


                    #Count the very short/partially-attend sessions
                    very_short_any=('very_short_any', 'max'),
                    total_very_short=('very_short_sum', 'sum'),
                    partial_any=('partial_any', 'max'),
                    total_partial=('partial_sum', 'sum'),

                    #others
                    days_present_w=('is_present_day', 'sum'),
                    days_observed_w=('date', 'nunique'),

                    #general info
                    total_attended_minutes_w=('minutes_attended_day', 'sum'),
                    total_planned_minutes_w=('total_duration_day', 'sum')

                )
                ).reset_index()

weekly_stats

,SID,week_start,late_max,early_max,total_late_minutes,total_early_minutes,late_mean,early_mean,attended_ratio_mean,attended_ratio_week,very_short_any,total_very_short,partial_any,total_partial,days_present_w,days_observed_w,total_attended_minutes_w,total_planned_minutes_w
0,00000006,2025-02-17,0.0,0.0,0.0,14.0,0.000000,7.000,0.872881,0.876033,0,0,1,1,1,1,106,121
1,00000006,2025-02-24,12.0,12.0,15.0,59.0,1.875000,9.875,0.760938,0.761308,0,0,1,3,4,4,338,430
2,00000006,2025-03-03,46.0,46.0,87.0,139.0,10.875000,17.375,0.492652,0.491510,1,3,1,6,3,4,241,489
3,00000006,2025-03-10,0.0,0.0,0.0,26.0,0.000000,13.000,0.778552,0.776860,0,0,1,1,1,1,94,121
4,00000006,2025-03-17,0.0,0.0,0.0,72.0,0.000000,9.000,0.833031,0.834568,0,0,1,3,4,4,404,484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46134,25000396,2025-05-12,2.0,2.0,2.0,0.0,0.500000,0.000,0.942281,0.942281,0,0,0,0,4,4,228,242
46135,25000396,2025-05-19,0.0,0.0,0.0,0.0,0.000000,0.000,0.970968,0.970968,0,0,0,0,4,4,235,242
46136,25000506,2025-05-12,13.0,13.0,13.0,0.0,3.250000,0.000,0.919492,0.919492,0,0,1,1,4,4,214,233
46137,25000506,2025-05-19,6.0,6.0,9.0,0.0,2.250000,0.000,0.907190,0.907190,0,0,0,0,4,4,217,239


#### Fill in the missing weeks of each student

In [35]:
HOLIDAYS = ['2024-09-02', '2024-09-03', '2024-11-05', '2024-11-11', '2024-11-28', '2024-11-29',
            '2024-12-23', '2024-12-24','2024-12-25','2024-12-26','2024-12-27','2024-12-28','2024-12-29','2024-12-30',
            '2024-12-31','2025-01-01','2025-01-02', '2025-01-20','2025-01-25','2025-01-26','2025-01-27','2025-01-28',
            '2025-01-29','2025-01-30','2025-01-31','2025-02-01','2025-02-02','2025-02-17','2025-04-05','2025-04-06',
            '2025-04-07','2025-04-30','2025-05-01','2025-05-26']

In [81]:
def fill_missing_weeks(weekly_df, sid_col="SID", week_col="week_start"):
    out=[]
    
    for sid, g in weekly_df.groupby(sid_col, sort=False):
        g = g.sort_values(week_col)
        if g.empty:
            continue
        
        start = g[week_col].min()
        end = g[week_col].max()

        full_weeks = pd.date_range(start, end, freq="W-MON")

        have = pd.DatetimeIndex(g[week_col])
        missing = full_weeks.difference(have)

        if len(missing):
            add = pd.DataFrame({sid_col: sid, week_col: pd.to_datetime(missing)})
            g=pd.concat([g,add], ignore_index=True, sort=False)

        out.append(g)

    res = pd.concat(out, ignore_index=True)
    res = res.sort_values([sid_col, week_col]).reset_index(drop=True)
    return res

In [ ]:
weekly_stats1 = fill_missing_weeks(weekly_stats, sid_col="SID", week_col="week_start")

count_cols = ['days_present_w', 'days_observed_w', 'total_attended_minutes_w', 'total_planned_minutes_w']
for c in count_cols:
    if c in count_cols:
        weekly_stats1[c] = weekly_stats1[c].fillna(0).astype(int)

weekly_stats1

#weekly_stats1.to_csv('filled_weeks.csv', index=False)

,SID,week_start,late_max,early_max,total_late_minutes,total_early_minutes,late_mean,early_mean,attended_ratio_mean,attended_ratio_week,very_short_any,total_very_short,partial_any,total_partial,days_present_w,days_observed_w,total_attended_minutes_w,total_planned_minutes_w
0,00000006,2025-02-17,0.0,0.0,0.0,14.0,0.000000,7.000,0.872881,0.876033,0.0,0.0,1.0,1.0,1,1,106,121
1,00000006,2025-02-24,12.0,12.0,15.0,59.0,1.875000,9.875,0.760938,0.761308,0.0,0.0,1.0,3.0,4,4,338,430
2,00000006,2025-03-03,46.0,46.0,87.0,139.0,10.875000,17.375,0.492652,0.491510,1.0,3.0,1.0,6.0,3,4,241,489
3,00000006,2025-03-10,0.0,0.0,0.0,26.0,0.000000,13.000,0.778552,0.776860,0.0,0.0,1.0,1.0,1,1,94,121
4,00000006,2025-03-17,0.0,0.0,0.0,72.0,0.000000,9.000,0.833031,0.834568,0.0,0.0,1.0,3.0,4,4,404,484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53427,25000396,2025-05-12,2.0,2.0,2.0,0.0,0.500000,0.000,0.942281,0.942281,0.0,0.0,0.0,0.0,4,4,228,242
53428,25000396,2025-05-19,0.0,0.0,0.0,0.0,0.000000,0.000,0.970968,0.970968,0.0,0.0,0.0,0.0,4,4,235,242
53429,25000506,2025-05-12,13.0,13.0,13.0,0.0,3.250000,0.000,0.919492,0.919492,0.0,0.0,1.0,1.0,4,4,214,233
53430,25000506,2025-05-19,6.0,6.0,9.0,0.0,2.250000,0.000,0.907190,0.907190,0.0,0.0,0.0,0.0,4,4,217,239


## Transform Engagement Dataset

In [90]:
engage = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\student-dropout-analysis\data_processed\final_goiten.xlsx", dtype={'SID':str})
engage

,SID_full,SID,SID_5digit,Name,Grade,Call,percent_response,success_call,no_resp_calls,Successful,No_response
0,1904-0019,19040019,40019,Hoàng Gia An,Grade 9,0,0.0,0,0,NaN,NaN
1,1904-0019,19040019,40019,Hoàng Gia An,EFL 69,0,0.0,0,0,NaN,NaN
2,1904-0020,19040020,40020,Hoàng Khải Lâm,Grade 9,0,0.0,0,0,NaN,NaN
3,1904-0020,19040020,40020,Hoàng Khải Lâm,EFL 69,0,0.0,0,0,NaN,NaN
4,1904-0024,19040024,40024,Lê Hà Thu,Grade 9,0,0.0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4431,2500-0268,25000268,268,Han Bao Nguyen,Grade 4,0,0.0,0,0,NaN,NaN
4432,2500-0288,25000288,288,Thanh Nguyen,Grade K,0,0.0,0,0,NaN,NaN
4433,2500-0319,25000319,319,Khang Duy Doan,Grade 2,0,0.0,0,0,NaN,NaN
4434,2500-0331,25000331,331,Hy Nghien Hua,Grade 1,0,0.0,0,0,NaN,NaN


In [94]:
engagement = engage.copy() 

def parse_dd_mm_list_to_weeks(s, start_year=2024):
    """Return a list of Monday week_start (W-MON) Timestamps from a 'dd/mm, dd/mm, ...' string."""
    if pd.isna(s) or str(s).strip()=="":
        return []
    parts = [p.strip() for p in str(s).split(",") if p.strip()]
    dts = pd.to_datetime(parts, format="%d/%m", errors="coerce", dayfirst=True)
    out = []
    for dt in dts:
        if pd.isna(dt): 
            continue
        m = int(dt.month)
        y = start_year if 6 <= m <= 12 else start_year + 1
        d = pd.Timestamp(year=y, month=int(dt.month), day=int(dt.day))
        out.append(pd.Period(d, freq="W-SUN").start_time)
    return out

# Build (SID, week_start) -> called_count_w ON THE FLY (no permanent table)
rows = []
for sid, s in engagement[["SID", "Successful"]].itertuples(index=False):
    for ws in parse_dd_mm_list_to_weeks(s):
        rows.append((sid, ws))

if rows:
    called_counts = (pd.DataFrame(rows, columns=["SID", "week_start"])
                     .value_counts()
                     .rename("called_count_w")
                     .reset_index())
else:
    called_counts = pd.DataFrame(columns=["SID", "week_start", "called_count_w"])


called_counts

,SID,week_start,called_count_w
0,24001070,2024-12-09,6
1,24001070,2024-10-28,6
2,20023177,2024-08-19,6
3,22002271,2024-06-24,6
4,20037981,2025-02-24,6
...,...,...,...
11363,22002046,2025-05-05,1
11364,22002046,2025-05-19,1
11365,22002047,2024-06-03,1
11366,22002047,2024-09-16,1


In [98]:
rows = []
for sid, s in engagement[["SID", "No_response"]].itertuples(index=False):
    for ws in parse_dd_mm_list_to_weeks(s):
        rows.append((sid, ws))

if rows:
    called_failed_counts = (pd.DataFrame(rows, columns=["SID", "week_start"])
                     .value_counts()
                     .rename("called_failed_w")
                     .reset_index())
else:
    called_failed_counts = pd.DataFrame(columns=["SID", "week_start", "called_failed_w"])


called_failed_counts

,SID,week_start,called_failed_w
0,24001456,2024-11-04,3
1,24001293,2024-10-07,3
2,24001827,2024-10-14,2
3,22000288,2024-08-12,2
4,22003235,2024-07-15,2
...,...,...,...
802,22003683,2024-10-21,1
803,22003683,2024-11-25,1
804,22003707,2025-05-19,1
805,22003890,2024-06-03,1


In [99]:
weekly_stats_all = (weekly_stats1
                .merge(called_counts, on=["SID","week_start"], how="left")
                .merge(called_failed_counts, on=['SID', 'week_start'], how='left'))

# Weeks with no engagement → 0 calls
weekly_stats_all["called_count_w"] = weekly_stats_all["called_count_w"].fillna(0).astype(int)
weekly_stats_all["called_failed_w"] = weekly_stats_all["called_failed_w"].fillna(0).astype(int)



weekly_stats_all

,SID,week_start,late_max,early_max,total_late_minutes,total_early_minutes,late_mean,early_mean,attended_ratio_mean,attended_ratio_week,very_short_any,total_very_short,partial_any,total_partial,days_present_w,days_observed_w,total_attended_minutes_w,total_planned_minutes_w,called_count_w,called_failed_w
0,00000006,2025-02-17,0.0,0.0,0.0,14.0,0.000000,7.000,0.872881,0.876033,0.0,0.0,1.0,1.0,1,1,106,121,0,0
1,00000006,2025-02-24,12.0,12.0,15.0,59.0,1.875000,9.875,0.760938,0.761308,0.0,0.0,1.0,3.0,4,4,338,430,0,0
2,00000006,2025-03-03,46.0,46.0,87.0,139.0,10.875000,17.375,0.492652,0.491510,1.0,3.0,1.0,6.0,3,4,241,489,0,0
3,00000006,2025-03-10,0.0,0.0,0.0,26.0,0.000000,13.000,0.778552,0.776860,0.0,0.0,1.0,1.0,1,1,94,121,0,0
4,00000006,2025-03-17,0.0,0.0,0.0,72.0,0.000000,9.000,0.833031,0.834568,0.0,0.0,1.0,3.0,4,4,404,484,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53427,25000396,2025-05-12,2.0,2.0,2.0,0.0,0.500000,0.000,0.942281,0.942281,0.0,0.0,0.0,0.0,4,4,228,242,0,0
53428,25000396,2025-05-19,0.0,0.0,0.0,0.0,0.000000,0.000,0.970968,0.970968,0.0,0.0,0.0,0.0,4,4,235,242,0,0
53429,25000506,2025-05-12,13.0,13.0,13.0,0.0,3.250000,0.000,0.919492,0.919492,0.0,0.0,1.0,1.0,4,4,214,233,0,0
53430,25000506,2025-05-19,6.0,6.0,9.0,0.0,2.250000,0.000,0.907190,0.907190,0.0,0.0,0.0,0.0,4,4,217,239,0,0


In [100]:
weekly_stats_all.to_csv('weekly_stats_all.csv', index=False)

## Add Quiz Data

In [104]:
canvas = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\student-dropout-analysis\data_processed\final_canvas.xlsx", dtype={'SID': str})
canvas

,ID,SID,Student,Section,Quiz,Score,Source_File,Quiz_Clean
0,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch1 (79507),21.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch1
1,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch10 (79508),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch10
2,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch11 (79509),17.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch11
3,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch12 (79506),18.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch12
4,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch13 (79505),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch13
...,...,...,...,...,...,...,...,...
394874,12104,22002539,Sophie - 22002539,Grade 3 Math F2024,Unposted Current Score,90.00,2025-09-06T0353_Grades-Grade_3_Math_F2024,Unposted Current Score
394875,12104,22002539,Sophie - 22002539,Grade 2 Humanities F2024,Unposted Final Score,34.89,2025-09-06T0351_Grades-Grade_2_Humanities_F2024,Unposted Final Score
394876,12104,22002539,Sophie - 22002539,Grade 2 Math F2024,Unposted Final Score,21.05,2025-09-06T0352_Grades-Grade_2_Math_F2024,Unposted Final Score
394877,12104,22002539,Sophie - 22002539,Grade 3 Humanities F2024,Unposted Final Score,0.00,2025-09-06T0353_Grades-Grade_3_Humanities_F2024,Unposted Final Score


In [ ]:
#Check the quiz scores and their possible values
out = canvas.groupby('Quiz_Clean')['Score'].agg(
    n_rows='size',
    n_no_null='count'
)

out['no_nan_pct'] = out['n_no_null']/out['n_rows']
out = out.sort_values(by='no_nan_pct')
out

,n_rows,n_no_null,no_nan_pct
Quiz_Clean,,,
�Exam Unposted Final Score,99,0,0.0
Migrated Quizzes Current Score,210,0,0.0
Math Vocabulary Review,203,0,0.0
Instructions and Practice - no grade Unposted Final Score,178,0,0.0
BrainPOP Unposted Final Score,168,0,0.0
...,...,...,...
Exam Final Points,168,168,1.0
Exam Current Points,168,168,1.0
Essays Unposted Final Score,315,315,1.0


In [125]:
#Remove the quizzes that have 0 students doing - potentially an optional quiz
mask_keep = canvas.groupby('Quiz_Clean')['Score'].transform(lambda s:s.notna().any())
canvas_filtered = canvas[mask_keep].copy()
canvas_filtered

,ID,SID,Student,Section,Quiz,Score,Source_File,Quiz_Clean
0,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch1 (79507),21.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch1
1,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch10 (79508),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch10
2,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch11 (79509),17.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch11
3,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch12 (79506),18.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch12
4,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch13 (79505),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch13
...,...,...,...,...,...,...,...,...
394874,12104,22002539,Sophie - 22002539,Grade 3 Math F2024,Unposted Current Score,90.00,2025-09-06T0353_Grades-Grade_3_Math_F2024,Unposted Current Score
394875,12104,22002539,Sophie - 22002539,Grade 2 Humanities F2024,Unposted Final Score,34.89,2025-09-06T0351_Grades-Grade_2_Humanities_F2024,Unposted Final Score
394876,12104,22002539,Sophie - 22002539,Grade 2 Math F2024,Unposted Final Score,21.05,2025-09-06T0352_Grades-Grade_2_Math_F2024,Unposted Final Score
394877,12104,22002539,Sophie - 22002539,Grade 3 Humanities F2024,Unposted Final Score,0.00,2025-09-06T0353_Grades-Grade_3_Humanities_F2024,Unposted Final Score


In [ ]:
#remove the quizzes that have under 20 people doing - potentially optional quizzes
MIN_PARTICIPANTS = 20

mask_keep = (canvas.groupby("Quiz_Clean")["Score"]
                   .transform(lambda s: s.notna().sum()) >= MIN_PARTICIPANTS)

canvas_filtered = canvas_filtered[mask_keep].copy()
canvas_filtered

C:\Users\Admin\AppData\Local\Temp\ipykernel_6944\2137449139.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  canvas_filtered = canvas_filtered[mask_keep].copy()


,ID,SID,Student,Section,Quiz,Score,Source_File,Quiz_Clean
0,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch1 (79507),21.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch1
1,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch10 (79508),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch10
2,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch11 (79509),17.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch11
3,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch12 (79506),18.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch12
4,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch13 (79505),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch13
...,...,...,...,...,...,...,...,...
394874,12104,22002539,Sophie - 22002539,Grade 3 Math F2024,Unposted Current Score,90.00,2025-09-06T0353_Grades-Grade_3_Math_F2024,Unposted Current Score
394875,12104,22002539,Sophie - 22002539,Grade 2 Humanities F2024,Unposted Final Score,34.89,2025-09-06T0351_Grades-Grade_2_Humanities_F2024,Unposted Final Score
394876,12104,22002539,Sophie - 22002539,Grade 2 Math F2024,Unposted Final Score,21.05,2025-09-06T0352_Grades-Grade_2_Math_F2024,Unposted Final Score
394877,12104,22002539,Sophie - 22002539,Grade 3 Humanities F2024,Unposted Final Score,0.00,2025-09-06T0353_Grades-Grade_3_Humanities_F2024,Unposted Final Score


In [135]:
SEM1_ALIASES = [
    r"\bF(20\d{2})\b",          # F2024
    r"\bFall\s*(20\d{2})\b",    # Fall 2024
    r"\bSY(20\d{2})\b",         # SY2025 
]

SEM2_ALIASES = [
    r"\bSP(20\d{2})\b",         # SP2025
    r"\bSpring\s*(20\d{2})\b",  # Spring 2025
]

def extract_semester(text: str):
    """
    Returns: (semester_num, semester_code)
      - semester_num: 1 for Fall/SY, 2 for Spring, np.nan if unknown
      - semester_code: 'FYYYY' or 'SPYYYY' (normalized), or None if unknown
    """
    if not isinstance(text, str) or not text.strip():
        return np.nan, None

    t = text.strip()

    # Try semester 1 patterns (Fall/F/SY)
    for pat in SEM1_ALIASES:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            year = m.group(1)
            # Normalize all sem1 to FYYYY per your mapping
            return 1, f"F{year}"

    # Try semester 2 patterns (Spring/SP)
    for pat in SEM2_ALIASES:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            year = m.group(1)
            return 2, f"SP{year}"

    # No match
    return np.nan, None

# Apply to your DataFrame
def add_semester_columns(df, title_col="Section"):
    out = df.copy()
    res = out[title_col].apply(extract_semester)
    out["semester_num"]  = res.apply(lambda x: x[0])
    out["semester_code"] = res.apply(lambda x: x[1])
    return out

semester_col_added = add_semester_columns(canvas_filtered, title_col="Section")
semester_col_added["semester_code"] = semester_col_added["semester_code"].replace({"F2025": "F2024"})

semester_col_added

,ID,SID,Student,Section,Quiz,Score,Source_File,Quiz_Clean,semester_num,semester_code
0,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch1 (79507),21.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch1,1,F2024
1,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch10 (79508),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch10,1,F2024
2,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch11 (79509),17.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch11,1,F2024
3,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch12 (79506),18.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch12,1,F2024
4,613,19040019,Hoàng Gia An - 19040019,AP Biology F2024,AP BIO ch13 (79505),16.00,2025-09-06T0350_Grades-AP_Biology_F2024,AP BIO ch13,1,F2024
...,...,...,...,...,...,...,...,...,...,...
394874,12104,22002539,Sophie - 22002539,Grade 3 Math F2024,Unposted Current Score,90.00,2025-09-06T0353_Grades-Grade_3_Math_F2024,Unposted Current Score,1,F2024
394875,12104,22002539,Sophie - 22002539,Grade 2 Humanities F2024,Unposted Final Score,34.89,2025-09-06T0351_Grades-Grade_2_Humanities_F2024,Unposted Final Score,1,F2024
394876,12104,22002539,Sophie - 22002539,Grade 2 Math F2024,Unposted Final Score,21.05,2025-09-06T0352_Grades-Grade_2_Math_F2024,Unposted Final Score,1,F2024
394877,12104,22002539,Sophie - 22002539,Grade 3 Humanities F2024,Unposted Final Score,0.00,2025-09-06T0353_Grades-Grade_3_Humanities_F2024,Unposted Final Score,1,F2024


In [137]:
quiz_stats = (semester_col_added.groupby(['SID', 'Section']).
agg(
    semester_code=('semester_code', 'first'),
    n_quiz=('Quiz_Clean', 'nunique'),
    n_quiz_done=('Score', lambda x: x.notna().sum()),
    score_mean=('Score', 'mean')
)).reset_index()

quiz_stats

,SID,Section,semester_code,n_quiz,n_quiz_done,score_mean
0,19040019,AP Biology F2024,F2024,40,39,42.563077
1,19040019,AP Biology SP2025,SP2025,37,37,46.438919
2,19040020,AP Biology F2024,F2024,40,40,44.524750
3,19040020,AP Biology SP2025,SP2025,37,38,54.812632
4,19040024,AP Biology F2024,F2024,40,35,43.031429
...,...,...,...,...,...,...
9223,25000432,AP Biology SP2025,SP2025,37,8,0.000000
9224,25000432,AP English Language and Composition SP2025,SP2025,28,8,0.000000
9225,25000432,AP PreCalculus SP2025,SP2025,66,10,0.000000
9226,25000506,Grade 1 Humanities Spring 2025,SP2025,30,19,64.526316


In [140]:
quiz_stats['score_pctile'] = (
    quiz_stats.groupby('Section')['score_mean']
    .rank(pct=True)
)

quiz_stats['completion_rate'] = (
    quiz_stats['n_quiz_done'] / quiz_stats['n_quiz']
)

quiz_stats['completion_rate'] = quiz_stats['completion_rate'].clip(upper=1)

quiz_stats

,SID,Section,semester_code,n_quiz,n_quiz_done,score_mean,score_pctile,completion_rate
0,19040019,AP Biology F2024,F2024,40,39,42.563077,0.685714,0.975000
1,19040019,AP Biology SP2025,SP2025,37,37,46.438919,0.820144,1.000000
2,19040020,AP Biology F2024,F2024,40,40,44.524750,0.794286,1.000000
3,19040020,AP Biology SP2025,SP2025,37,38,54.812632,0.949640,1.000000
4,19040024,AP Biology F2024,F2024,40,35,43.031429,0.702857,0.875000
...,...,...,...,...,...,...,...,...
9223,25000432,AP Biology SP2025,SP2025,37,8,0.000000,0.241007,0.216216
9224,25000432,AP English Language and Composition SP2025,SP2025,28,8,0.000000,0.240876,0.285714
9225,25000432,AP PreCalculus SP2025,SP2025,66,10,0.000000,0.240625,0.151515
9226,25000506,Grade 1 Humanities Spring 2025,SP2025,30,19,64.526316,0.429952,0.633333


In [142]:
agg_quiz = (quiz_stats.groupby(['SID', 'semester_code'])
                              .agg(
                                  avg_score_mean=('score_mean', 'mean'),
                                  avg_score_min=('score_mean', 'min'),
                                  score_pctile_mean=('score_pctile', 'mean'),
                                  score_pctile_min=('score_pctile', 'min'),
                                  completion_rate_mean=('completion_rate', 'mean'),
                                  completion_rate_min=('completion_rate', 'min')
                              )).reset_index()

agg_quiz

,SID,semester_code,avg_score_mean,avg_score_min,score_pctile_mean,score_pctile_min,completion_rate_mean,completion_rate_min
0,19040019,F2024,42.563077,42.563077,0.685714,0.685714,0.975000,0.975000
1,19040019,SP2025,46.438919,46.438919,0.820144,0.820144,1.000000,1.000000
2,19040020,F2024,44.524750,44.524750,0.794286,0.794286,1.000000,1.000000
3,19040020,SP2025,54.812632,54.812632,0.949640,0.949640,1.000000,1.000000
4,19040024,F2024,55.741714,43.031429,0.713788,0.702857,0.937500,0.875000
...,...,...,...,...,...,...,...,...
3844,25000392,SP2025,81.790210,67.272727,0.654589,0.449275,0.672464,0.478261
3845,25000396,SP2025,14.409091,0.000000,0.216667,0.104762,0.258333,0.150000
3846,25000425,SP2025,0.000000,0.000000,0.114826,0.107558,0.168666,0.130435
3847,25000432,SP2025,0.000000,0.000000,0.240836,0.240625,0.217815,0.151515


In [143]:
agg_quiz.columns

Index(['SID', 'semester_code', 'avg_score_mean', 'avg_score_min',
       'score_pctile_mean', 'score_pctile_min', 'completion_rate_mean',
       'completion_rate_min'],
      dtype='object')